In [8]:
#lancer le modéle et vérifier la résultat
import cv2
import numpy as np
import pandas as pd
from keras.models import model_from_json


emotion_dict = {0: "Happy", 1: "Neutral", 2: "Sad"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

# start the webcam feed
#cap = cv2.VideoCapture(0)

# pass here your video path
# you may download one from here : https://www.pexels.com/video/three-girls-laughing-5273028/
cap = cv2.VideoCapture(0)
resulta = []
i=0
while True:
    count_0 = 0
    count_1 = 0
    count_2 = 0
    
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        maxindex = int(np.argmax(emotion_prediction))
        if maxindex == 0:
            count_0 = 1
        if maxindex == 1:
            count_1 =1
        if maxindex == 2:
            count_2 =1
   
        resulta.append([i,count_0,count_1,count_2])
        i=i+1

    
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or i==200 :
        break
#le modéle s'arréte si on clique sur le boutton 'q' ou si le nomre de frame égale 200
cap.release()
cv2.destroyAllWindows()



Loaded model from disk


In [7]:
#collecter la resultat en dataframe
import pandas as pd
resulta = pd.DataFrame(resulta,columns=["frame","Happy","Neutral","Surprised"])
resulta

,frame,Happy,Neutral,Surprised
0,0,0,1,0


In [ ]:
#crée un fichier excel qui contient la résultat obtenu
resulta.to_excel("Nom&tom3.xlsx")